# Using FastAI for creating a Waste Classifier

Fastai is a deep learning library which provides high-level components that can quickly and easily provide state-of-the-art results in standard deep learning domains. 

It has two main design goals: 
1. To be approachable and rapidly productive
2. To be also configurable. 

## Waste Classifier
The aim is to build a model for waste classification that identifies among the different classes: 
- cardboard
- compost
- glass
- metal
- paper
- plastic
- trash

This machine learning model will help people to improve their decision when classifying trash   

## Imports

In [ ]:
from fastai.vision.all import load_learner, vision_learner, resnet50, RandomResizedCrop, ToTensor, aug_transforms
from fastai.metrics import accuracy

from fastai.vision.data import ImageDataLoaders

import warnings
warnings.filterwarnings('ignore')

!pip install mlxtend

## Load the data

The data is already splitted in train and test folders. Inside each folder contains one folder for each class. Those images were obtained using Bing searcher using the api HTTP.   
Those images have been manually cleaned, removing the ones that were not useful or were in the wrong category. 
The data has been divided into two sets train and test sets.  

In [ ]:
BASE_DIR = "."
DATASET_DIR = f"{BASE_DIR}/dataset_splits"
TEST_PHOTOS_DIR = f"{BASE_DIR}/test-photos"

In [ ]:
!ls $DATASET_DIR

In [ ]:
!ls $DATASET_DIR/train/glass

## Data augmentation

Apply data augmentation to the training data, keeping 10% the for validation set. Data augmentation process apply some transformations (flip, rotate) to the original images to generate more data to train the model. Plus, helps to avoid overfitting, making the training set less specific. 

The validation set is used to track the model error while training. 


In [ ]:
import os

data = ImageDataLoaders.from_folder(
    DATASET_DIR,
    train ='train',
    shuffle=True,
    valid_pct=0.1,
    item_tfms =[ToTensor,RandomResizedCrop(224, min_scale=0.35)],
    batch_tfms=aug_transforms(flip_vert=True, batch=True, max_rotate=30.0),
)

Let's show some of the data for the validation and training sets.

In [ ]:
data.valid_ds.items[:3]

In [ ]:
data.train_ds.items[:3]

In [ ]:
data.train.show_batch()

In [ ]:
data.valid.show_batch()

In [ ]:
data.vocab

## Training the model using resnet50

ResNet34 is a convolutional neural network(CNN) that has 34 layers. It has been already trained with images from the ImageNet database. It classifies 1000 object from very broad categories, such as pencil or animals. The input size of the network is 224x224. 

This network can be reused to train other model. 

In [ ]:
learn = vision_learner(data, resnet50, metrics=accuracy)
learn.fine_tune(20)
MODEL_FILENAME = "result-resnet50.pkl"
learn.export(fname=f'{BASE_DIR}/{MODEL_FILENAME}')

## Metrics
Now, let's calculate the accuracy metric for the test set. 
The test set refers to a group of data that has been separated form the training data and the model does not know. 
It contains original images without any transformation. It will reflect how well the model will behave in the real life, when classifying new images.

Let's see what is the result for some photos that we took with the cellphone and check what is the output of the model. 

In [ ]:
import utils 

We can see that has good accuracy, so we decided to use this model for our final solution

In [ ]:
learn_loaded = load_learner(f'{BASE_DIR}/{MODEL_FILENAME}')
utils.show_predictions(learn_loaded, TEST_PHOTOS_DIR)                      
utils.print_results(data.vocab, learn_loaded, f"{DATASET_DIR}/test", 'classification_matrix_resnet50.png')